In [2]:
import pandas as pd
r = pd.read_csv('result.csv', encoding='utf8')
r.head()

,Object,ObjectAnchor,Predicate,PredicateId,Subject,SubjectAnchor,Text,WikipediaLink,WikipediaTitle,_id,Answer
0,Кім Чен Ір,Кім Чен Ір,батько,P22,Кім Чен Ин,Кім Чен Ин,Кім Чен Ин (нар. 1983) — син колишнього диктат...,https://uk.wikipedia.org/wiki/Кім,Кім,97e85e9c-5968-47d0-93e5-17a41114b50f,weak
1,Самудрагупта,Самудрагупта,батько,P22,Чандрагупта ІІ,Чандрагупта ІІ,"При ЧандрагуптіІ, внукові Шрі-Гупти, що правив...",https://uk.wikipedia.org/wiki/Династія Гуптів,Династія Гуптів,613060fa-811c-463a-ab1c-712619568c66,has
2,Тит Флавій Петрон,Тит Флавій Петрон,батько,P22,Тит Флавій Сабін (батько Веспасіана),Тит Флавій Сабін,Походив із середніх італійських прошарків; дід...,https://uk.wikipedia.org/wiki/Веспасіан,Веспасіан,a71ce5d2-63ea-45af-9ce7-959d3573d950,donthas
3,Шунь-цзун (династія Тан),Шунь-цзуна,батько,P22,Сянь-цзун (династія Тан),Сянь-цзун,У Китаї розпочалося коротке правління Шунь-цзу...,https://uk.wikipedia.org/wiki/805,805,8568c7e1-5e84-4541-a693-29acb5cfa056,not
4,Шунь-цзун (династія Тан),Шунь-цзуна,батько,P22,Сянь-цзун (династія Тан),Сянь-цзуна,"Відомо, що працював за імператорів Шунь-цзуна ...",https://uk.wikipedia.org/wiki/Дай Сун,Дай Сун,2b8a672e-31cf-4fc8-9fd2-570e5eaf3def,weak


In [97]:
r = r[r['Answer']!='not']
len(r)

2234

In [171]:
r['Answer'].value_counts()

has        963
donthas    920
weak       351
Name: Answer, dtype: int64

In [169]:
row = r.iloc[1960]
text = row['Text']
print text

Єдиний син надвірного коронного хорунжого Прокопа Сенявського та його дружини Анни Єфросини Ходкевич, єдиний внук Адама Єроніма Сенявського (старшого) та Катерини Сенявської.


In [170]:
from tokenize_uk import *

ttext = u'Єдиний син надвірного коронного хорунжого Прокопа Сенявського та його дружини Анни Єфросини Ходкевич, єдиний внук Адама Єроніма Сенявського (старшого) та Катерини Сенявської.'

tokens = tokenize_text(text)
#tokens = [t for tt in tokens for sentence in tt for t in sentence]
print ', '.join(tokens[0][0])

Єдиний, син, надвірного, коронного, хорунжого Прокопа, Сенявського та, його, дружини Анни, Єфросини, Ходкевич, ,, єдиний, внук Адама, Єроніма, Сенявського, (, старшого, ), та Катерини, Сенявської, .


In [151]:
print tokens[4]

хорунжого Прокопа


In [130]:
import re
objectAnchor = row['ObjectAnchor']
def detect_anchor(tokens, anchor):
    found = False
    index = 0
    for i in range(len(tokens)):
        token = tokens[i]
        pattern = re.escape(anchor[index]) + u"[а|ом|у]?"
        match = re.match(pattern, token, flags=re.U|re.L)
        
        if(found):
            if(match is not None):
                index += 1
            else:
                index = 0
                found = False
                pattern = re.escape(anchor[index]) + u"[а|ом]?"
                match = re.match(pattern, token, flags=re.U|re.L)
        
        if(not found and match is not None):
            start = i
            index += 1
            found = True
            
        if(index == len(anchor)):
            end = i + 1
            yield range(start, end)
            index = 0
            found = False
            


def extract_from_tokens_by_range(tokens, r):
    return ' '.join(tokens[r[0]:r[-1]+1])
    
detection = list(detect_anchor(tokens, tokenize_words(objectAnchor)))
print extract_from_tokens_by_range(tokens, detection[0])

князь Володимир Великий


In [131]:
subjectAnchor = row['SubjectAnchor']
detection = list(detect_anchor(tokens, tokenize_words(subjectAnchor)))
detection

[[50, 51]]

In [132]:
print extract_from_tokens_by_range(tokens, detection[0])

Ярослав Мудрий


In [133]:
def show_row_anchors_and_text(row, verbose=False):
    text = row['Text']
    if(verbose):
        print text
    tokens = tokenize_text(text)
    tokens = [t for tt in tokens for sentence in tt for t in sentence]
    detection1 = list(detect_anchor(tokens, tokenize_words(row['SubjectAnchor'])))
    if(verbose):
        for d in detection1:
            print extract_from_tokens_by_range(tokens, d)
    detection2 = list(detect_anchor(tokens, tokenize_words(row['ObjectAnchor'])))
    if(verbose):
        for d in detection2:
            print extract_from_tokens_by_range(tokens, d)
    return (detection1, detection2)
        
show_row_anchors_and_text(r.iloc[0], True)

Кім Чен Ин (нар. 1983) — син колишнього диктатора КНДР Кім Чен Іра, онук Кім Ір Сена
Кім Чен Ин
Кім Чен Іра


([[0, 1, 2]], [[13, 14, 15]])

In [134]:
show_row_anchors_and_text(r.iloc[1], True)

При ЧандрагуптіІ, внукові Шрі-Гупти, що правив в 320 — близько 340 і прийняв імператорський титул, почалося утворення держави Гуптів. Самудрагупта і Чандрагупта ІІ Вікрамадітья (правив близько 380–414) підкорили всю Північну Індію, крім Кашміру, Західного Пенджабу і Раджпутани.
Чандрагупта ІІ
Самудрагупта


([[27, 28]], [[25]])

In [135]:
show_row_anchors_and_text(r.iloc[2], True)

Походив із середніх італійських прошарків; дід Тит Флавій Петрон служив центуріоном у Помпея, батько Тит Флавій Сабін — збирач податків в Азії.
Тит Флавій Сабін
Тит Флавій Петрон


([[16, 17, 18]], [[7, 8, 9]])

In [136]:
positions = r.apply(show_row_anchors_and_text, axis=1)

In [143]:
err = []
for i in range(len(positions)):
    try:
        p = positions.iloc[i]
    except:
        err.append(i)
    if(len(p[0]) == 0):
        err.append(i)
    if(len(p[1]) == 0):
        err.append(i)
err

[834, 1303, 1960, 1960]

In [146]:
r.iloc[1960]

Object                                           Катерина Сенявська
ObjectAnchor                                    Катерини Сенявської
Predicate                                                      мати
PredicateId                                                     P25
Subject                                Прокоп Сенявський (хорунжий)
SubjectAnchor                                   Прокопа Сенявського
Text              Єдиний син надвірного коронного хорунжого Прок...
WikipediaLink     https://uk.wikipedia.org/wiki/Велика Севастянівка
WikipediaTitle                                  Велика Севастянівка
_id                            aab548e5-7e34-4f59-8aae-664038bba262
Answer                                                      donthas
Name: 2049, dtype: object

In [148]:
show_row_anchors_and_text(r.iloc[1960],True)

Єдиний син надвірного коронного хорунжого Прокопа Сенявського та його дружини Анни Єфросини Ходкевич, єдиний внук Адама Єроніма Сенявського (старшого) та Катерини Сенявської.


([], [])